In [70]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [71]:
driver = webdriver.Chrome('D:\Downloads/chromedriver')
driver.implicitly_wait(3)
driver.get('http://www.op.gg/summoner/userName=ktmata')

In [72]:
driver.find_element_by_xpath('//a[@data-type="soloranked"]').click()

In [94]:
while True:
    game_more_button = driver.find_elements_by_css_selector('div.GameMoreButton.Box')
    if game_more_button == []:
        break
    game_more_button[0].click()
    time.sleep(5)

In [95]:
game_detail_button = driver.find_elements_by_css_selector('a.Button.MatchDetail')
for a in game_detail_button:
    a.click()
    time.sleep(5)

In [98]:
html = driver.page_source
PlayerData = BeautifulSoup(html, 'html.parser')
print(PlayerData.prettify())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [110]:
team_raw = PlayerData.find_all('tbody', {'class':'Content'})
game_raw = PlayerData.find_all('thead', {'class':'Header'})

win_lose = []
champion_list = []
name_list = []
kill_list = []
death_list = []
assist_list = []

for gline in game_raw:
    game_result_raw = gline.find_all('span', {'class':'GameResult'})
    
    game_result = []
    for gresult in game_result_raw:
        gresult = gresult.text.strip()
        game_result.append(gresult)
    win_lose.append(game_result)

for line in team_raw:
    teamdata = []
    team_champion_raw = line.find_all('td', {'class':'ChampionImage Cell'})
    team_name_raw = line.find_all('td', {'class':'SummonerName Cell'})
    team_kill_raw = line.find_all('span',{'class':'Kill'})
    team_death_raw = line.find_all('span',{'class':'Death'})
    team_assist_raw = line.find_all('span',{'class':'Assist'})
    
    team_champion = []
    for champ in team_champion_raw:
        champ = champ.text.strip()
        champ = champ[:champ.find('\n')]
        team_champion.append(champ)
    champion_list.append(team_champion)

    team_name = []
    for name in team_name_raw:
        name = name.text.strip()
        team_name.append(name)
    name_list.append(team_name)

    team_kill = []
    team_death = []
    team_assist = []
    team_kill_participation = []
    
    for kills in team_kill_raw:
        kills = kills.text.strip()
        team_kill.append(kills)
    kill_list.append(team_kill)

    for deaths in team_death_raw:
        deaths = deaths.text.strip()
        team_death.append(deaths)
    death_list.append(team_death)

    for assists in team_assist_raw:
        assists = assists.text.strip()
        team_assist.append(assists)
    assist_list.append(team_assist)
    
win_lose = win_lose[1:]

KDAsorted = []
matchorder = 1
matchnumber = 1
for order in range(int((len(win_lose)+1)/2)):
    matchnumber = matchnumber-1
    for i in range(5):
        team_member = []
        team_member.append(matchorder)
        team_member.append(win_lose[matchnumber][0])
        team_member.append(champion_list[matchnumber][i])
        team_member.append(name_list[matchnumber][i])
        team_member.append(int(kill_list[matchnumber][i]))
        team_member.append(int(death_list[matchnumber][i]))
        team_member.append(int(assist_list[matchnumber][i]))
        KDAsorted.append(team_member)
    matchnumber += 1
    for i in range(5):
        team_member = []
        team_member.append(matchorder)
        team_member.append(win_lose[matchnumber][0])
        team_member.append(champion_list[matchnumber][i])
        team_member.append(name_list[matchnumber][i])
        team_member.append(int(kill_list[matchnumber][i]))
        team_member.append(int(death_list[matchnumber][i]))
        team_member.append(int(assist_list[matchnumber][i]))
        KDAsorted.append(team_member)
    matchnumber += 2
    matchorder += 1

kda_data = pd.DataFrame(KDAsorted, columns=['MatchNumber','GameResult', 'Champion', 'Nickname', 'Kill', 'Death', 'Assist'])
kda_data['KDA'] = (kda_data['Kill'] + kda_data['Assist'])/ kda_data['Death']
kda_data.head()

,MatchNumber,GameResult,Champion,Nickname,Kill,Death,Assist,KDA
0,1,승리,나르,쌀국수뚝붸기,4,2,8,6.000000
1,1,승리,세주아니,정글노바,4,0,8,inf
2,1,승리,말자하,KT MATA,3,1,7,10.000000
3,1,승리,미스 포츈,Afreeca M0wgli,5,5,8,2.600000
4,1,승리,탐 켄치,진쟤승,4,1,10,14.000000
